# Final Project

## 1. Programming environment

In [1]:
import pandas as pd
import requests
import json

## 2. Data Collection

Our group chose the website https://www.communitybenefitinsight.org/?page=info.data_api to collect our data. This website contains information about hospitals across the United States. 
The Community Benefit Insight data API allows us to retrieve the following types of data:
+ Hospital data (optionally filtered by state)
+ Detailed data about a single hospital

### 2.1 Retrieve hospital data


In [3]:
url = 'https://www.communitybenefitinsight.org/api/get_hospitals.php'
data = requests.get(url).text
hospital_data = json.loads(data)
attribute_names = list(hospital_data[0].keys())
data = {}
for attribute in attribute_names:
    data[attribute] = [item[attribute] for item in hospital_data]
df = pd.DataFrame.from_dict(data)
df.to_csv('./data/full_hospital.csv', index=False)
print(1)

### 2.2 Retrieve detailed data from every hospital

To retrieve data for each hospital, we use https://www.communitybenefitinsight.org/api/get_hospital_data.php?hospital_id= + ID (ID is the hospital ID)

Get attributes for the dataset 

In [12]:
url = 'https://www.communitybenefitinsight.org/api/get_hospital_data.php?hospital_id=1'
data = requests.get(url).text
web = json.loads(data)
attribute = list(web[-1].keys())
attribute

['hospital_data_id',
 'fiscal_yr',
 'data_ein',
 'data_name',
 'form_990_filed_part_grp_ret_f',
 'form_990_num_fac_oper',
 'form_990_name',
 'form_990_address',
 'form_990_city',
 'form_990_state',
 'form_990_zip',
 'tot_func_exp',
 'tot_revenue',
 'tot_comm_bnfts',
 'chrty_care',
 'unreim_medcd',
 'unreim_costs',
 'comm_hlth_impr_svcs_comm_bnft_oper',
 'hlth_prof_educ',
 'subsd_hlth_svcs',
 'rsrch',
 'cash_inknd_contrib_comm_grps',
 'comm_bldg',
 'comm_bldg_actvs',
 'comm_bldg_actvs_physimprvhse',
 'comm_bldg_actvs_econdevlp',
 'comm_bldg_actvs_cmntysuprt',
 'comm_bldg_actvs_envrnimprv',
 'comm_bldg_actvs_ldrdevlp',
 'comm_bldg_actvs_cltnbldg',
 'comm_bldg_actvs_htlhimprvadvcy',
 'comm_bldg_actvs_wrkfrcdevlp',
 'comm_bldg_actvs_other',
 'comm_bldg_actvs_physimprvhse_pct',
 'comm_bldg_actvs_econdevlp_pct',
 'comm_bldg_actvs_cmntysuprt_pct',
 'comm_bldg_actvs_envrnimprv_pct',
 'comm_bldg_actvs_ldrdevlp_pct',
 'comm_bldg_actvs_cltnbldg_pct',
 'comm_bldg_actvs_htlhimprvadvcy_pct',
 'comm_

Create a dictionary to store value for each attribute. The dictionary has one extra key to store the hospital id.

In [13]:
single_data = {}
single_data['hospital_id'] = []
for name in attribute:
    single_data[name] = []

There are total 3491 hospitals in the dataset but the website only allowed us to make 100 requests per week. So I had to change my VPN  to retrieve information about 100 hospitals at a time.

In [ ]:
for id in range(1, 100):
    new_url = 'https://www.communitybenefitinsight.org/api/get_hospital_data.php?hospital_id=' + str(id)
    new_data = requests.get(new_url).text
    json_data = json.loads(new_data)
    for name in attribute:
        single_data[name].append(json_data[-1][name])
    single_data['hospital_id'].append(id)
    # print(id)

Save the data as csv format

In [ ]:
df = pd.DataFrame.from_dict(single_data)
df.to_csv('./data/test.csv', index=False)

Finally I will concatenate that dataset to the first one based on the hospital_id attribute.

In [26]:
first_df = pd.read_csv('./data/full_hospital.csv')
second_df = pd.read_csv('./data/test.csv')
raw_df = pd.merge(first_df, second_df, on='hospital_id')
raw_df.to_csv('./data/raw_data.csv', index = False)
raw_df.head(5)

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,...,pctttlexp_econdevlp,pctttlexp_cmntysuprt,pctttlexp_envrnimprv,pctttlexp_ldrdevlp,pctttlexp_cltnbldg,pctttlexp_htlhimprvadvcy,pctttlexp_wrkfrcdevlp,pctttlexp_other,pctttlexp_total,updated_dt_y
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,1039,...,0.0,0.0001,0.0,0.0,0.0,0.0001,0.0017,0.0,0.0019,"November 20, 2023"
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,1073,...,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,"November 20, 2023"
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,1117,...,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,"November 20, 2023"
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,1073,...,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,"November 20, 2023"
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,1019,...,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,"November 20, 2023"


## 3. Data Preprocessing & Exploration

### 3.1 Data Preprocessing

In [52]:
raw_df = pd.read_csv('./data/raw_data.csv')

#### Check for duplicate row in data 

In [53]:
index = raw_df.index
detectDupSeries = index.duplicated(keep='first')
num_duplicated_rows = detectDupSeries.sum()

if num_duplicated_rows == 0:
    print(f"Data have no duplicated line.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate your raw data.!")

Data have no duplicated line.!


#### Choosing attribute

There are 161 features in the dataframe which is too many so we will choose only essential features in the dataframe to explore

First find the columns that has NaN value

In [54]:
nan_count = raw_df.isna().sum()

columns_with_NaN_value = nan_count[nan_count > 0]
# columns = columns_with_NaN_value.index.tolist()
# columns
len(columns_with_NaN_value)

41

There are 41 columns in the dataset that has NaN value. So we will delete columns that 20% of it is NaN values.

In [55]:
chosen_columns = nan_count[nan_count > 698]
del_columns = chosen_columns.index.tolist()
for name in del_columns:
    del raw_df[name]
raw_df.shape
# raw_df.to_csv('./data/first.csv')

(3491, 144)

We see there are a lot of columns in the dataframe that has a lot of 0 value. So we will see those that has the most 0 value count.

In [56]:
columns_with_0 = raw_df.columns[(raw_df == 0).sum() > 0]
columns_with_0

Index(['tot_func_exp', 'tot_revenue', 'tot_comm_bnfts', 'chrty_care',
       'unreim_medcd', 'unreim_costs', 'comm_hlth_impr_svcs_comm_bnft_oper',
       'hlth_prof_educ', 'subsd_hlth_svcs', 'rsrch',
       ...
       'pctttlexp_physimprvhse', 'pctttlexp_econdevlp', 'pctttlexp_cmntysuprt',
       'pctttlexp_envrnimprv', 'pctttlexp_ldrdevlp', 'pctttlexp_cltnbldg',
       'pctttlexp_htlhimprvadvcy', 'pctttlexp_wrkfrcdevlp', 'pctttlexp_other',
       'pctttlexp_total'],
      dtype='object', length=101)

We will delete columns that 20% of it is 0.

In [57]:
drop_columns = raw_df.columns[(raw_df == 0).sum() > 698]
raw_df = raw_df.drop(columns=drop_columns)
raw_df.shape

(3491, 68)

The dataframe now only has 68 columns. Next our group will eliminate attributes that our group think it is irrelevant.

In [58]:
del raw_df['hospital_org_id'], raw_df['updated_dt_x'], raw_df['updated_dt_y']
del raw_df['zip_code'], raw_df['fips_state_and_county_code'], raw_df['data_ein'], raw_df['name_cr'], raw_df['data_name'], raw_df['hospital_bed_size']
del raw_df['form_990_name'], raw_df['form_990_address'], raw_df['form_990_city'], raw_df['form_990_state'], raw_df['form_990_zip'], raw_df['hospital_data_id']
del raw_df['per_capita_inc_qrt'], raw_df['med_hsehld_inc_qrt'], raw_df['percent_ppl_pov_qrt'], raw_df['percent_ppl_65_yrs_old_without_hlth_insr_qrt'], raw_df['unemp_rate_16_yrs_older_qrt']

Many columns are just flag, so we will select carefully attributes to delete

In [59]:
columns_with_Y = raw_df.columns[(raw_df == 'Y').sum() > 0]
for x in columns_with_Y:
    print(x)
    print(raw_df[x].value_counts()['Y'])

chrch_affl_f
347
urban_location_f
2288
children_hospital_f
101
memb_counc_teach_hosps_f
1099
form_990_filed_part_grp_ret_f
1481
sole_comm_prov_f
25
written_fncl_asst_policy_f
3484
used_fed_pov_gdlns_det_elig_free_disc_care_f
3473
chna_cond_comm_hlth_needs_assmt_f
3316
chna_defined_comm_served_f
3314
chna_took_into_acct_input_ppl_rep_broad_intrst_comm_served_f
3313
chna_cond_w_mult_hosp_facilities_f
2004
chna_made_wide_avail_pub_f
3318
hosp_adp_imp_stg_addr_hlth_needs_comm_f
3284
st_impl_aca_medcd_exp_cvrg_adlts_w_incs_up_138pct_fpl_f
2205
st_law_req_hosp_rpt_comm_bnfts_f
2480
paper_return_f
65


In [60]:
columns_with_N = raw_df.columns[(raw_df == 'N').sum() > 0]
for x in columns_with_N:
    print(x)
    print(raw_df[x].value_counts()['N'])

chrch_affl_f
3144
urban_location_f
1203
children_hospital_f
3390
memb_counc_teach_hosps_f
2392
form_990_filed_part_grp_ret_f
2010
sole_comm_prov_f
3466
written_fncl_asst_policy_f
1
used_fed_pov_gdlns_det_elig_free_disc_care_f
12
chna_cond_comm_hlth_needs_assmt_f
80
chna_took_into_acct_input_ppl_rep_broad_intrst_comm_served_f
9
chna_cond_w_mult_hosp_facilities_f
1318
chna_made_wide_avail_pub_f
4
hosp_adp_imp_stg_addr_hlth_needs_comm_f
26
st_impl_aca_medcd_exp_cvrg_adlts_w_incs_up_138pct_fpl_f
1286
st_law_req_hosp_rpt_comm_bnfts_f
1011
paper_return_f
3426


We drop the flag columns that are divided so uneven.

In [61]:
drop_columns_Y = raw_df.columns[(raw_df == 'Y').sum() > 3491*0.8]
raw_df = raw_df.drop(columns=drop_columns_Y)

drop_columns_N = raw_df.columns[(raw_df == 'N').sum() > 3491*0.8]
raw_df = raw_df.drop(columns=drop_columns_N)

raw_df.shape

(3491, 37)

Next we need to fill in NaN value.

In [62]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 37 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   hospital_id                                              3491 non-null   int64  
 1   ein                                                      3491 non-null   int64  
 2   name                                                     3491 non-null   object 
 3   street_address                                           3491 non-null   object 
 4   city                                                     3491 non-null   object 
 5   state                                                    3491 non-null   object 
 6   hospital_bed_count                                       3491 non-null   int64  
 7   urban_location_f                                         3491 non-null   object 
 8   memb_counc_teach_hosps_f    

In [63]:
nan_count = raw_df.isna().sum()
columns_with_NaN_value = nan_count[nan_count > 0]
columns_with_NaN_value = columns_with_NaN_value.index.to_list()
num_df = raw_df.select_dtypes(exclude='object')
fill_col = num_df.columns.intersection(columns_with_NaN_value)
for name in fill_col:
    raw_df[name] = raw_df[name].fillna(raw_df[name].median())

In [64]:
nan_count = raw_df.isna().sum()
columns_with_NaN_value = nan_count[nan_count > 0]
columns_with_NaN_value

county                                  1
chna_cond_w_mult_hosp_facilities_f    169
dtype: int64

We filled in NaN value with the median of the column except for the firt and last column whose type are float.

In [65]:
raw_df.to_csv('./data/final_data.csv', index=False)

### 3.2 Data exploration

In [66]:
df = pd.read_csv('./data/final_data.csv')

#### How many rows and how many columns does the raw data have?

In [67]:
shape = df.shape
print(f"The data has {shape[0]} rows and {shape[1]} columns.")

The data has 3491 rows and 37 columns.


#### What data type does each column currently have?

In [68]:
df.dtypes

hospital_id                                                  int64
ein                                                          int64
name                                                        object
street_address                                              object
city                                                        object
state                                                       object
hospital_bed_count                                           int64
urban_location_f                                            object
memb_counc_teach_hosps_f                                    object
medicare_provider_number                                     int64
county                                                      object
fiscal_yr                                                    int64
form_990_filed_part_grp_ret_f                               object
form_990_num_fac_oper                                        int64
tot_func_exp                                                 i

The data type looks suitable for us to explore.

#### What does each column mean?

- hospital_id: Internal hospital ID
- ein: Hospital EIN
- name: Hospital name
- street_address: Street address
- city: City
- state: State
- hospital_bed_count: Number of beds
- urban_location_f: Urban location flag
- memb_counc_teach_hosps_f: Member of the Council of Teaching Hospitals flag
- county: Hospital county
- fiscal_yr: Fiscal Year
- form_990_filed_part_grp_ret_f: Form 990 was filed as part of a group return
- form_990_num_fac_oper: Form 990 number of facilities operated
- tot_func_exp: Total functional expenses
- tot_revenue: Total revenue
- tot_comm_bnfts: Total community benefits
- chrty_care: Charity care
- unreim_medcd: Unreimbursed Medicaid
- unreim_costs: Unreimbursed costs
- comm_hlth_impr_svcs_comm_bnft_oper: Community health improvement services and community benefit operations
- bad_debt: Bad debt
- mdcre_shrtfl: Medicare Shortfall (negative value indicates surplus)
- tot_comm_bnfts_tot_func_exp_pct: Total community benefits as % of total functional expenses
- chrty_care_tot_func_exp_pct: Charity care as % of total functional expenses
- unreim_medcd_tot_func_exp_pct: Unreimbursed Medicaid as % of total functional expenses
- comm_hlth_impr_svcs_comm_bnft_oper_tot_func_exp_pct: Community health improvement services and community benefit operations as % of total functional expenses
- bad_debt_tot_func_exp_pct: Bad debt as % of total functional expenses
- mdcre_shrtfl_tot_func_exp_pct: Medicare Shortfall as % of total functional expenses (negative value indicates surplus)
- rat_pat_care_npat_care_comm_bnfts: Ratio of Patient Care to Non-patient Care Community Benefits
- chna_cond_w_mult_hosp_facilities_f: CHNA conducted with multiple hospital facilities
- per_capita_inc: per capita income
- med_hsehld_inc: median household income
- percent_ppl_pov: percent persons in poverty
- percent_ppl_65_yrs_old_without_hlth_insr: percent persons <65 years old without health insurance
- unemp_rate_16_yrs_older: unemployment rate, 16 years and older
- st_impl_aca_medcd_exp_cvrg_adlts_w_incs_up_138pct_fpl_f: State implemented ACA Medicaid expansion covering adults with incomes up to 138% FPL
- st_law_req_hosp_rpt_comm_bnfts_f: State law requiring hospital to report community benefits

#### For each column with numeric data type, how are the values distributed?

In [69]:
num_col_info_df = df.select_dtypes(exclude='object')
del num_col_info_df['hospital_id']
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)
    # raise NotImplementedError()

def median(df):
    return (df.quantile(0.5)).round(1)
    # raise NotImplementedError()

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)
    # raise NotImplementedError()

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)
    # raise NotImplementedError()

num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])
num_col_info_df

,ein,hospital_bed_count,medicare_provider_number,fiscal_yr,form_990_num_fac_oper,tot_func_exp,tot_revenue,tot_comm_bnfts,chrty_care,unreim_medcd,...,unreim_medcd_tot_func_exp_pct,comm_hlth_impr_svcs_comm_bnft_oper_tot_func_exp_pct,bad_debt_tot_func_exp_pct,mdcre_shrtfl_tot_func_exp_pct,rat_pat_care_npat_care_comm_bnfts,per_capita_inc,med_hsehld_inc,percent_ppl_pov_pct,percent_ppl_65_yrs_old_without_hlth_insr_pct,unemp_rate_16_yrs_older
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
min,10130427.0,2.0,10007.0,2010.0,1.0,0.000000e+00,-1.615266e+08,-1.877057e+06,-7511212.0,-4.359135e+07,...,-100000.000000,-0.018872,-1.174136,-81.982677,-3372.579913,0.0,0.0,0.0,0.0,0.0
lower_quartile,352528741.0,32.0,141333.0,2021.0,1.0,6.311005e+07,6.592734e+07,4.182806e+06,464754.0,7.136190e+05,...,0.900000,0.000000,0.900000,-4.300000,0.900000,44490.0,51720.0,9.4,7.0,4.0
median,476028103.0,114.0,261335.0,2021.0,1.0,2.940063e+08,3.093186e+08,2.235120e+07,3236355.0,7.412280e+06,...,3.300000,0.100000,2.300000,-1.700000,2.500000,51141.0,60070.0,12.4,9.5,5.0
upper_quartile,741356589.0,275.0,390074.5,2021.0,5.0,1.292430e+09,1.423750e+09,1.267807e+08,18159097.0,4.264596e+07,...,6.000000,0.300000,4.700000,0.100000,6.800000,60138.0,71799.0,15.4,13.2,6.3
max,990269825.0,3060.0,670309.0,2021.0,43.0,2.881186e+10,3.222840e+10,1.411503e+09,333132504.0,1.089481e+09,...,742.167399,42593.461997,100000.000000,100000.000000,22391.329545,191220.0,155362.0,44.7,35.8,19.8
